<a href="https://colab.research.google.com/github/aaolcay/Convolutional-Neural-Network-Examples/blob/main/Fashion_mnist_KerasAPI_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Authenticate Drive Account**

This is because we will upload the model (used in `Fashion_mnist_Transfer_Learning.ipynb`) so that we can build a Functional model with the same architecture.

In [160]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


**See what we have in our folder**

In [161]:
import os
os.chdir('/gdrive/My Drive/Convolutional Neural Networks')
!ls

Fashion_mnist_KerasAPI_model.ipynb
Fashion_mnist_Transfer_Learning.ipynb
mnist_dataset_Sequential_model.ipynb
model_fashion-mnist_cnn_train2_epoch24.h5


**Import Libraries**

`keras` -> To create DL (here we use CNN) model

`keras.datasets` -> Calling a dataset ready from Kaggle (Keras library)

`keras.models` -> `load_model` -> For transfer learning (loading a model ready)

`numpy` -> Save your life if you are using arrays

`matplotlib.pyplot` -> To plot images, graphs etc.



In [162]:
import tensorflow.keras as keras
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt

**Load Model**

See the summary of model so that we can build the same architecture by Keras API. 

In [163]:
model = keras.models.load_model('/gdrive/My Drive/Convolutional Neural Networks/model_fashion-mnist_cnn_train2_epoch24.h5')
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 12, 12, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 9216)              0         
                                                                 
 dense_1 (Dense)             (None, 128)               1179776   
                                                      

**Load Data**

In [164]:
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train = x_train[...,np.newaxis]
x_test = x_test[...,np.newaxis]
input_shape = (x_train.shape[1],x_train.shape[2],x_train.shape[3])

In [165]:
# Make labels categorical
num_class = 10
y_train = keras.utils.to_categorical(y_train, num_class)
y_test = keras.utils.to_categorical(y_test, num_class)

**Investigating Model**



In [166]:
# see activation functions in each layer
for i in range(len(model.layers)):
  if model.layers[i].count_params() != 0: # check if the layer is of parameters
                                          # as this means that we have an activation 
                                          # function in this layer
                                           
    print(f'Layer: {i}, Activation function: {model.layers[i].activation}')

Layer: 0, Activation function: <function relu at 0x7fcf8e34e790>
Layer: 1, Activation function: <function relu at 0x7fcf8e34e790>
Layer: 5, Activation function: <function relu at 0x7fcf8e34e790>
Layer: 7, Activation function: <function softmax at 0x7fcf8e346d30>


In [167]:
# see the kernel sizes in convolutional layers
print(f'Kernel sizes are: {model.layers[0].kernel_size}, {model.layers[1].kernel_size}')

Kernel sizes are: (3, 3), (3, 3)


In [168]:
# see if there is padding and stride size in the convolutional layers
print(f'paddings: {model.layers[0].padding}, {model.layers[1].padding}')
print(f'Stride sizes are: {model.layers[0].strides}, {model.layers[1].strides}')

paddings: valid, valid
Stride sizes are: (1, 1), (1, 1)


Paddings are valid in the convolutional layers, so there is no padding!

In [169]:
# See what maxpooling has
print(f'Max Pooling Layer Summary:\npool size: {model.layers[2].pool_size}\nstrides: {model.layers[2].strides}\npadding: {model.layers[2].padding}')

Max Pooling Layer Summary:
pool size: (2, 2)
strides: (2, 2)
padding: valid


In [170]:
# See the rate of dropout
print(f'Probabilities of dropouts:\nLayer {3}: {model.layers[3].rate}\nLayer {6}: {model.layers[6].rate}')

Probabilities of dropouts:
Layer 3: 0.25
Layer 6: 0.5


**Build Functional Model**

In [171]:
model_input = keras.Input(shape=input_shape, name='model_input')
model_conv_1 = keras.layers.Conv2D(32,
                                  kernel_size = (3,3),
                                  strides = (1,1),
                                  activation='relu',
                                  padding='valid',
                                  name = 'conv_layer_1'   
                                  )(model_input)

model_conv_2 = keras.layers.Conv2D(64,
                                  kernel_size = (3,3),
                                  strides = (1,1),
                                  activation='relu',
                                  padding='valid',
                                  name = 'conv_layer_2'   
                                  )(model_conv_1)

model_max_pool = keras.layers.MaxPool2D((2,2), # grid size
                                     strides=(2,2), # strides
                                     padding='valid',# applies max pooling to whole input
                                     name='max_pool_1')(model_conv_2)

dropout1 = keras.layers.Dropout(0.25, name='dropout1')(model_max_pool) # see above, why we chose 0.25

flatten1 = keras.layers.Flatten(name='flatten_1')(dropout1)

dense1 = keras.layers.Dense(128, 
                                     activation='relu', 
                                     name='Dense_1')(flatten1)

dropout2 = keras.layers.Dropout(0.5, name='dropout2')(dense1) # see above, why we chose 0.25

output = keras.layers.Dense(10, # number of classes
                                  activation='softmax', 
                                  name='model_output')(dropout2)

In [172]:
# Make this network a model
model_API = keras.Model(model_input, output, name='conv_model_API_aao')

In [173]:
model_API.summary()

Model: "conv_model_API_aao"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_input (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 conv_layer_1 (Conv2D)       (None, 26, 26, 32)        320       
                                                                 
 conv_layer_2 (Conv2D)       (None, 24, 24, 64)        18496     
                                                                 
 max_pool_1 (MaxPooling2D)   (None, 12, 12, 64)        0         
                                                                 
 dropout1 (Dropout)          (None, 12, 12, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 9216)              0         
                                                                 
 Dense_1 (Dense)             (None, 128)        

As seen from the summary of the model built using Keras API, number of parameters, output shapes are the same as that called model (`model_fashion-mnist_cnn_train2_epoch24.h5`).

In [174]:
# see the model's compiling functions
print(f'Models optimizer is {model.optimizer}')
#print(f'Learning rate in optimizer is {model.optimizer.learning_rate}')
print(f'Models loss function is {model.loss}')
#print(f'Models metric is {model.metrics}')

Models optimizer is <keras.optimizers.optimizer_v2.adadelta.Adadelta object at 0x7fceebaeea90>
Models loss function is <function categorical_crossentropy at 0x7fcf94f42c10>


In [175]:
model_API.compile(optimizer=keras.optimizers.Adadelta(), loss=keras.losses.CategoricalCrossentropy())

In [176]:
history = model_API.fit(x_train, y_train, epochs=24, validation_data=(x_test, y_test), shuffle=True, verbose =1)

Epoch 1/24
1875/1875 [==============================] - 19s 5ms/step - loss: 14.5471 - val_loss: 1.4032
Epoch 2/24
1875/1875 [==============================] - 9s 5ms/step - loss: 3.1510 - val_loss: 1.6406
Epoch 3/24
1875/1875 [==============================] - 10s 5ms/step - loss: 2.2118 - val_loss: 1.5882
Epoch 4/24
1875/1875 [==============================] - 10s 5ms/step - loss: 1.9942 - val_loss: 1.4784
Epoch 5/24
1875/1875 [==============================] - 10s 5ms/step - loss: 1.8680 - val_loss: 1.3548
Epoch 6/24
1875/1875 [==============================] - 11s 6ms/step - loss: 1.7763 - val_loss: 1.2359
Epoch 7/24
1875/1875 [==============================] - 10s 5ms/step - loss: 1.6790 - val_loss: 1.1388
Epoch 8/24
1875/1875 [==============================] - 9s 5ms/step - loss: 1.6148 - val_loss: 1.0585
Epoch 9/24
1875/1875 [==============================] - 10s 5ms/step - loss: 1.5473 - val_loss: 1.0044
Epoch 10/24
1875/1875 [==============================] - 10s 5ms/step - lo

In [178]:
test_error = model_API.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 1s 3ms/step - loss: 0.6900


**Evaluate the Accuracy of Model**


In [181]:
T = []
F = []

for iteration in range(200):
  x = x_test[iteration,:,:,:]
  x = x[np.newaxis, ...]
  prediction = model.predict(x)
  prediction = np.argmax(prediction)
  if prediction == np.where(y_test[iteration] == 1):
    T.append(prediction)
  else:
    F.append(prediction)

1/1 [==============================] - 0s 20ms/step


In [182]:
accuracy = len(T)/(len(T)+len(F))
print(f'Accuracy of the model is: {100*accuracy}%')

Accuracy of the model is: 86.0%


As seen we get the same accuracy as the model ready resulted in [here](https://github.com/aaolcay/Convolutional-Neural-Network-Examples/blob/main/Fashion_mnist_Transfer_Learning.ipynb).

Mission completed, if you have any question just let me know: olcayazzam@gmail.com.

***If you want to use my codes please refer to me: Abdullah Olcay (University of Southampton).***